### Define configs (Hyper-parameters)

In [1]:
import argparse

parser = argparse.ArgumentParser()
# Dataset
parser.add_argument('--dataset', type=str, default='LiquidSugar', help='dataset name')
parser.add_argument('--target', type=str, default='yield_flow', help='target')
parser.add_argument('--task_type', type=str, default='forecasting', help='task type: forecasting or prediction')
parser.add_argument('--collection_interval', type=int, default=15, help='Data collection interval (minutes)')
parser.add_argument('--n_stage', type=int, default=3, help='number of stages')
parser.add_argument('--seq_len', type=int, default=48, help='sequence length')
parser.add_argument('--pred_len', type=int, default=1, help='prediction length')
# Dataloader
parser.add_argument('--batch_size', type=int, default=64, help='pretrain batch size')
parser.add_argument('--n_workers', type=int, default=0, help='number of workers for DataLoader')
# Train set
parser.add_argument('--learning_rate', type=float, default=0.001, help='learning_rate')
parser.add_argument('--n_epoch', type=int, default=100, help='train_epoch')
parser.add_argument('--patience', type=int, default=10, help='early stopping patience')
# Model
parser.add_argument('--model_name', type=str, default='MultistageNet', help='model name')
parser.add_argument('--d_model', type=int, default=64, help='d_model of variable dimension')
parser.add_argument('--d_ff', type=int, default=256, help='feed foward dimension (d_model * 4)')
parser.add_argument('--kernel_size', type=int, default=3, help='number of head in attention')
parser.add_argument('--n_attention_heads', type=int, default=2, help='number of head in attention')
parser.add_argument('--n_temporal_layers', type=int, default=3, help='number of stack in DepthwiseCausalConv')
parser.add_argument('--n_multistage_layers', type=int, default=3, help='number of stack in Multistage Layer')
parser.add_argument('--n_regressor_layers', type=int, default=3, help='number of stack in Regressor')
parser.add_argument('--dropout_p', type=float, default=0.2, help='dropout rate')
parser.add_argument('--activation', type=str, default='GELU', help='activation function')
# Else
parser.add_argument('--seed', type=int, default=3, help='set seed')
parser.add_argument('--device_num', type=str, default='0', help='set gpu number')

args = parser.parse_args(args=[])

In [2]:
from utils.dotdict import dotdict
configs = dotdict(vars(args))
print(configs)

{'dataset': 'LiquidSugar', 'target': 'yield_flow', 'task_type': 'forecasting', 'collection_interval': 15, 'n_stage': 3, 'seq_len': 48, 'pred_len': 1, 'batch_size': 64, 'n_workers': 0, 'learning_rate': 0.001, 'n_epoch': 100, 'patience': 10, 'model_name': 'MultistageNet', 'd_model': 64, 'd_ff': 256, 'kernel_size': 3, 'n_attention_heads': 2, 'n_temporal_layers': 3, 'n_multistage_layers': 3, 'n_regressor_layers': 3, 'dropout_p': 0.2, 'activation': 'GELU', 'seed': 3, 'device_num': '0'}


### Define save-path

In [3]:
log_path = 'save/log/train_log_{}_{}_{}_{}.txt'.format(configs.model_name, configs.dataset, configs.seed, configs.seq_len)
model_save_path = 'save/model/{}_{}_{}_{}.pth'.format(configs.model_name, configs.dataset, configs.seed, configs.seq_len)
result_path = 'save/result/{}_{}_{}_{}.csv'.format(configs.model_name, configs.dataset, configs.seed, configs.seq_len)

### Load dataset

In [4]:
from utils.set_seed import set_seed
from utils.datautils import Load_liquidsugar_data

set_seed(configs.seed)
data, stage_vars, segment_index, slices, scaler = Load_liquidsugar_data(configs)

In [ ]:
print(data.head())

In [ ]:
print(stage_vars)

### Get data-loader

In [5]:
from utils.dataloader import get_dataloader

set_seed(configs.seed)
data_loader = {}
data_loader['train'] = get_dataloader(data, configs, segment_index, slices, flag='train', shuffle = True, drop_last=True)
data_loader['val'] = get_dataloader(data, configs, segment_index, slices, flag='val', shuffle = True, drop_last=True)
data_loader['test'] = get_dataloader(data, configs, segment_index, slices, flag='test')

### Define model

In [6]:
from model.MultistageNet import MultistageNet

model = MultistageNet(configs, stage_vars)

### Model train

In [7]:
import torch
from utils.trainer import train

set_seed(configs.seed)
best_model, val_metric = train(model, data_loader, configs, log_path)
torch.save(best_model, model_save_path)

Epoch: 1/100 | Train loss: 0.2430 | Valid loss: 0.2912 | Patience: 0 | Time per epoch: 6.62 (sec)
==> Find best valid at epoch 1 with valid loss of 0.2912
Epoch: 2/100 | Train loss: 0.1796 | Valid loss: 0.2530 | Patience: 0 | Time per epoch: 6.23 (sec)
==> Find best valid at epoch 2 with valid loss of 0.2530
Epoch: 3/100 | Train loss: 0.1692 | Valid loss: 0.2068 | Patience: 0 | Time per epoch: 6.32 (sec)
==> Find best valid at epoch 3 with valid loss of 0.2068
Epoch: 4/100 | Train loss: 0.1706 | Valid loss: 0.2156 | Patience: 1 | Time per epoch: 6.28 (sec)
Epoch: 5/100 | Train loss: 0.1663 | Valid loss: 0.2172 | Patience: 2 | Time per epoch: 6.13 (sec)
Epoch: 6/100 | Train loss: 0.1631 | Valid loss: 0.2183 | Patience: 3 | Time per epoch: 6.27 (sec)
Epoch: 7/100 | Train loss: 0.1619 | Valid loss: 0.2107 | Patience: 4 | Time per epoch: 6.37 (sec)
Epoch: 8/100 | Train loss: 0.1605 | Valid loss: 0.2058 | Patience: 0 | Time per epoch: 6.26 (sec)
==> Find best valid at epoch 8 with valid los

### Model test

In [8]:
from utils.trainer import test

model.load_state_dict(best_model)
test_metric = test(model, data_loader, configs)
print('MSE: {} | RMSE: {} | MAE: {}'.format(test_metric[0], test_metric[1], test_metric[2]))

MSE: 0.2453 | RMSE: 0.4953 | MAE: 0.2861


### Save results

In [9]:
from utils.write_results import write_results
results = {
    'val_loss': val_metric[0],
    'test_mse': test_metric[0],
    'test_rmse': test_metric[1],
    'test_mae': test_metric[2]
}

# write result
write_results(result_path, results, configs)